In [ ]:
##Python venv: search_agent_poc

In [1]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [1]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [2]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=99
end_page_index=480
pdf_enrichment_output_dir = './pdf_enriched_output/'
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename

In [3]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name,temperature=0.2)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

/home/matrix4284/mambaforge-pypy3/envs/search_agent_poc/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/matrix4284/mambaforge-pypy3/envs/search_agent_poc/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [4]:
#### Initial Load data (deserialized data)
####with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase1_entity_extraction_478_final.pickle', 'rb') as handle:
###    document_dict_deserialized = pickle.load(handle)

####If error happens start the load with intermediate index
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_phase1_relation_extraction_98_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)
    


In [5]:
document_dict_deserialized[2]

{'page': 91,
 'img_cnt': 0,
 'img_npy_lst': [],
 'text': '59\nIntroducing pods\n Having said that, do you think a multi-tier application consisting of a frontend\napplication server and a backend database should be configured as a single pod or as\ntwo pods?\nSPLITTING MULTI-TIER APPS INTO MULTIPLE PODS\nAlthough nothing is stopping you from running both the frontend server and the\ndatabase in a single pod with two containers, it isn’t the most appropriate way. We’ve\nsaid that all containers of the same pod always run co-located, but do the web server\nand the database really need to run on the same machine? The answer is obviously no,\nso you don’t want to put them into a single pod. But is it wrong to do so regardless? In\na way, it is.\n If both the frontend and backend are in the same pod, then both will always be\nrun on the same machine. If you have a two-node Kubernetes cluster and only this sin-\ngle pod, you’ll only be using a single worker node and not taking advantage of t

In [6]:
##Definitions of Individual Enrichment Modules######

# Function to reverse a string
def reverse(string):
    string = string[::-1]
    return string

def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


def extract_relationship_per_page(page_text,entity_lst):

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker\n
            Machine Learning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page related to a particular technology and extract all \n
            the possible relations between the list of entities provided as input along with the context/document.\n
            While extracting relations carry out the following steps as per the below rules for each : \n
            1.You must define each relation by means of a json which must have values for three keys source_entity,description,destination_entity.
                a.source_entity should have that entity who is the one performing some action that is the subject. \n
                b.destination_entity should have the entity who is the object on which the source entity carries out \n
                one more more actions.\n
                c.Description of the relation will contain a brief summary about what action was being carried by the \n
                source_entity on the destination_entity. \n\n
            2.Make Sure the extracted source_entity,destination_entity and Description are always \n
            within double quotes.\n
            3.After all the relations have been extracted collate them into a list of json.
            4.Make sure the json is terminated properly.
            Output should only contain the list of json and no other words or character or sentences. \n
            The out json list must be terminated properly.
            Here is the document page: \n\n {context} \n\n and the entities: \n\n {entities} \n\n""",
            input_variables=["context","entities"],
    )

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "context": page_text,"entities":entity_lst
        }
    )
    
    #print('Relationship Output from LLM"')
    #print(output)
    
    if '[' in output:
        json_output=output.split('[')[1]
        json_output='['+json_output
    else:    
        json_output='['+output
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    

    json_output=reverse(json_output)
    
    if ']' in json_output:
        json_output=json_output.split(']')[1]
        json_output=reverse(json_output)
        json_output= json_output + ']'
    else:
        json_output=reverse(json_output)
        json_output= json_output + ']'
        
    
    
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output

In [7]:
######This code enrcihes the PDF page by page and while doing so it fails after processing 20 to 50 pages####
######currently we are manually tackling this problem by making the pdf dict list into aglobal variable######
######After the llm fails we take the index of the last page done and save the pdf dict list uptil that ####
#####Then as of now i am manually restarting the by loading the saved pdf dict list as the source pdf and ###
#####Using the last done pdf page index +1 as the starting index.So (last Done page Index + 1) is the ######
###starting index###
####The maximum page has been given as 565 as that is the last page where the main content of the document###
####resides.In case of the book Kubernetes in action it is 565.##############################################
####If this is batch process then the last page can be manually seen and used as the variable else one can ####
#####also go for the pdf enrichment of all the pages of the book#############################################

###This manual thing needs to be automated by either langraph or by using agents framework##################

def enrich_page(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    entities_jsonl=document_dict_deserialized[page_idx]['entities']
    
    entity_lst=entity_collector_per_page(entities_jsonl)
    
    
    ##Relationship Extraction Enrichment
    if len(entity_lst)>0:
        page_relationship_lst_dict=extract_relationship_per_page(page_text,entity_lst)
        
        print("Page Relationship List Dictionary")
        print(page_relationship_lst_dict)
    
        document_dict_deserialized[page_idx]['relationships']=json.loads(page_relationship_lst_dict)
        print("Done for page "+str(page_idx))
        return page_idx
    else:
        print("there is no entity extracted from this page:"+str(page_idx))
        return page_idx
    

In [ ]:

for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        
        
        
        
    

Entity:
{'entity': 'Kubernetes', 'description': 'container orchestration system', 'category': 'software'}
Entity:
{'entity': 'DNS', 'description': 'domain name system', 'category': 'network'}
Entity:
{'entity': 'Pods', 'description': 'lightweight and ephemeral containers', 'category': 'application'}
Entity:
{'entity': 'Services', 'description': 'abstraction layer for accessing pods', 'category': 'software'}
Entity:
{'entity': 'kubectl', 'description': 'command-line tool for interacting with Kubernetes', 'category': 'software'}
Entity:
{'entity': 'exec', 'description': 'command for executing a command in a container', 'category': 'command'}
Entity:
{'entity': 'nslookup', 'description': 'command for looking up DNS records', 'category': 'command'}
Entity:
{'entity': 'headless services', 'description': "services that don't have an IP address", 'category': 'software'}
Entity:
{'entity': 'load balancing', 'description': 'distributing traffic across multiple pods', 'category': 'process'}
Enti

Page Relationship List Dictionary
[
    {
        "source_entity": "\"Kubernetes Resources\"",
        "description": "are managed by",
        "destination_entity": "\"kubectl apply\""
    },
    {
        "source_entity": "\"kubectl apply\"",
        "description": "is used to deploy",
        "destination_entity": "\"pod container\""
    },
    {
        "source_entity": "\"kubectl apply\"",
        "description": "creates a new instance of",
        "destination_entity": "\"Services\""
    },
    {
        "source_entity": "\"readiness probe\"",
        "description": "is used to check the health of",
        "destination_entity": "\"pod container\""
    },
    {
        "source_entity": "\"DNS\"",
        "description": "is used by",
        "destination_entity": "\"pod IPs\""
    },
    {
        "source_entity": "\"kubectl exec\"",
        "description": "allows to execute commands on",
        "destination_entity": "\"bash shell\""
    },
    {
        "source_entity": "\"kubec

Page Relationship List Dictionary
[{"source_entity": "ContentAgent", "description": "writes to", "destination_entity": "/var/"},
 {"source_entity": "Container: ContentAgent", "description": "contains", "destination_entity": "/var/"},
 {"source_entity": "Pod", "description": "hosts", "destination_entity": "Container: ContentAgent"},
 {"source_entity": "ContentAgent", "description": "writes to", "destination_entity": "/html/"},
 {"source_entity": "Container: WebServer", "description": "contains", "destination_entity": "/html/"},
 {"source_entity": "Pod", "description": "hosts", "destination_entity": "Container: WebServer"},
 {"source_entity": "ContentAgent", "description": "reads from", "destination_entity": "/htdocs/"},
 {"source_entity": "Container: ContentAgent", "description": "contains", "destination_entity": "/htdocs/"},
 {"source_entity": "Pod", "description": "hosts", "destination_entity": "Container: ContentAgent"},
 {"source_entity": "ContentAgent", "description": "reads from",

Page Relationship List Dictionary
[
  {
    "source_entity": "photonPersistentDisk",
    "description": "provides persistent storage for Photon containers",
    "destination_entity": "container"
  },
  {
    "source_entity": "flocker",
    "description": "manages and provisions persistent volumes for Flocker clusters",
    "destination_entity": "persistentVolumeClaim"
  },
  {
    "source_entity": "rbd",
    "description": "provides a block-level storage system for RBD volumes",
    "destination_entity": "persistentVolumeClaim"
  },
  {
    "source_entity": "cinder",
    "description": "manages and provisions persistent volumes for Cinder clusters",
    "destination_entity": "persistentVolumeClaim"
  },
  {
    "source_entity": "scaleIO",
    "description": "provides a block-level storage system for ScaleIO volumes",
    "destination_entity": "persistentVolumeClaim"
  },
  {
    "source_entity": "glusterfs",
    "description": "manages and provisions persistent volumes for GlusterFS cl

Page Relationship List Dictionary
[
    {
        "source_entity": "Kubernetes",
        "description": "manages",
        "destination_entity": "ReplicationController"
    },
    {
        "source_entity": "Kubernetes",
        "description": "uses",
        "destination_entity": "container"
    },
    {
        "source_entity": "Kubernetes",
        "description": "mounts",
        "destination_entity": "volumes"
    },
    {
        "source_entity": "ReplicationController",
        "description": "deploys",
        "destination_entity": "pod"
    },
    {
        "source_entity": "pod",
        "description": "uses",
        "destination_entity": "tmpfs filesystem"
    },
    {
        "source_entity": "pod",
        "description": "mounts",
        "destination_entity": "emptyDir"
    },
    {
        "source_entity": "Kubernetes",
        "description": "pulls from",
        "destination_entity": "Git repository"
    },
    {
        "source_entity": "Kubernetes",
        "descrip

...

Page Relationship List Dictionary
[
    {"source_entity": "kubectl", "description": "requests storage from", "destination_entity": "PersistentVolumeClaim"},
    {"source_entity": "kubectl", "description": "gets information about", "destination_entity": "claim"},
    {"source_entity": "kubectl", "description": "accesses resources of", "destination_entity": "resources"},
    {"source_entity": "Kubernetes", "description": "manages PersistentVolumeClaim", "destination_entity": "pvc"},
    {"source_entity": "Kubernetes", "description": "provides storage for", "destination_entity": "PersistentVolume"},
    {"source_entity": "kubectl", "description": "requests RWX access mode for", "destination_entity": "pv"},
    {"source_entity": "kubectl", "description": "requests RWO access mode for", "destination_entity": "ROX"},
    {"source_entity": "kubectl", "description": "requests ROX access mode for", "destination_entity": "storage"}
]
Done for page 123
Entity:
{'entity': 'PersistentVolume', 'desc

Page Relationship List Dictionary
[
    {
        "source_entity": "\"kubectl\"",
        "description": "uses",
        "destination_entity": "\"mongodb-pvc\""
    },
    {
        "source_entity": "\"kubectl\"",
        "description": "deletes",
        "destination_entity": "\"mongodb-pv\""
    },
    {
        "source_entity": "\"kubectl\"",
        "description": "retains",
        "destination_entity": "\"PersistentVolume\""
    },
    {
        "source_entity": "\"pod\"",
        "description": "uses",
        "destination_entity": "\"GCE Persistent Disks\""
    },
    {
        "source_entity": "\"kubernetes\"",
        "description": "manages",
        "destination_entity": "\"pod\""
    },
    {
        "source_entity": "\"kubectl\"",
        "description": "manipulates",
        "destination_entity": "\"PersistentVolumeClaim\""
    }
]
Done for page 126
Entity:
{'entity': 'PersistentVolume', 'description': 'a Kubernetes resource that represents a storage asset', 'category': 

Page Relationship List Dictionary
[
    {
        "source_entity": "\"kubectl describe\"",
        "description": "describe the properties of",
        "destination_entity": "\"storageClassName\""
    },
    {
        "source_entity": "\"gcloud compute disks list\"",
        "description": "list all available",
        "destination_entity": "\"disks\""
    },
    {
        "source_entity": "\"kubectl describe\"",
        "description": "describe the properties of",
        "destination_entity": "\"PersistentVolumeClaim\""
    },
    {
        "source_entity": "\"kubernetes.io/gce-pd provisioner\"",
        "description": "provision a new",
        "destination_entity": "\"StorageClass\""
    },
    {
        "source_entity": "\"StorageClass\"",
        "description": "define the characteristics of",
        "destination_entity": "\"ReadWriteOnce\""
    },
    {
        "source_entity": "\"kubectl get pvc\"",
        "description": "retrieve information about",
        "destination_enti

Page Relationship List Dictionary
[
    {
        "source_entity": "\"storageClassName\"",
        "description": "is associated with",
        "destination_entity": "\"pvc\""
    },
    {
        "source_entity": "\"kubectl\"",
        "description": "is used to manage",
        "destination_entity": "\"PersistentVolumeClaim\""
    },
    {
        "source_entity": "\"gcloud\"",
        "description": "is used to interact with",
        "destination_entity": "\"compute disks list\""
    },
    {
        "source_entity": "\"kubectl\"",
        "description": "is used to get information about",
        "destination_entity": "\"PV\""
    },
    {
        "source_entity": "\"kubectl\"",
        "description": "is used to get information about",
        "destination_entity": "\"PersistentVolume\""
    },
    {
        "source_entity": "\"pvc\"",
        "description": "is a type of",
        "destination_entity": "\"PersistentVolumeClaim\""
    },
    {
        "source_entity": "\"pv\"",
 

Page Relationship List Dictionary
[
    {
        "source_entity": "\"Kubernetes resources\"",
        "description": "are used to manage and orchestrate Docker containers",
        "destination_entity": "\"Docker containers\""
    },
    {
        "source_entity": "\"config file\"",
        "description": "is used to store configuration settings for a Kubernetes pod",
        "destination_entity": "\"pod\""
    },
    {
        "source_entity": "\"Git repository\"",
        "description": "is used as a source of Docker container images",
        "destination_entity": "\"container image\""
    },
    {
        "source_entity": "\"volume\"",
        "description": "is used to persist data for a Kubernetes pod",
        "destination_entity": "\"pod\""
    },
    {
        "source_entity": "\"Secret\"",
        "description": "is used to store sensitive information for a Kubernetes pod",
        "destination_entity": "\"pod\""
    },
    {
        "source_entity": "\"ConfigMaps\"",
      

In [10]:
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_phase1_relation_extraction_98_final.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
document_dict_deserialized[155]

In [ ]:
####The main contents of Kubernets In Action book is contained from page 87 to 565####
####Thats why we will only take the sliced portion i.e. from page 87 to 565 of the pdf dictionary list#####

In [48]:
# Load data (deserialize)
with open('pdf_content_dict_page_final.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [ ]:
# Store data (serialize in a pickle) upto page 102
with open('../pdf_enrichment/pdf_enriched_output/pdf_enriched_content_dict_stage1.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized, handle, protocol=pickle.HIGHEST_PROTOCOL)